# client instance 생성

In [38]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
# DEPLOYMENT_URL="http://localhost:2024/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [39]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': '0e746112-8537-43fe-b3f2-72aa514bb228', 'created_at': '2025-08-12T02:21:38.048555+00:00', 'updated_at': '2025-08-12T02:21:38.048555+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None, 'interrupts': {}, 'error': None}
0e746112-8537-43fe-b3f2-72aa514bb228


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [48]:
def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'agent' in node_name:
        #     if chunk.data['agent']['messages'][-1]["name"] != None:
            return True
    return False

def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True
            print(chunk.data['agent']['messages'][-1]['content'])

In [41]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [42]:

input = { "messages": "안녕"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
안녕하세요! 슬라이드 생성에 대해 도와드릴 수 있는 내용이 있을까요? 발표 주제나 필요한 내용에 대해서 말씀해 주세요.


In [43]:
input = { "messages": "제주도 여행을 주제로 슬라이드를 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
좋은 선택입니다! 제주도 여행에 관한 슬라이드를 만드는 데 있어 몇 가지 질문을 드릴게요:

1. **슬라이드의 목적**: 이 슬라이드는 개인적인 여행 계획을 위한 것인가요, 아니면 다른 사람에게 제주도를 소개하는 발표용인가요?

2. **포함하고 싶은 내용**: 특정 장소, 음식, 액티비티 등 꼭 다루고 싶은 주제가 있나요?

3. **슬라이드 수**: 몇 장의 슬라이드를 원하시나요?

4. **스타일이나 디자인**: 특별히 원하는 디자인이나 스타일이 있나요? 예를 들면, 밝고 색감 있는 스타일, 심플한 흑백 스타일 등 말이죠.

5. **타겟 audience**: 이 슬라이드를 보는 사람들에 대해 특별히 고려해야 할 사항이 있을까요?

이 정보를 토대로 슬라이드를 제작하겠습니다.


In [44]:
input = {"messages": "알아서 해줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
            print(chunk.data)
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########

{'agent': {'messages': [{'content': '', 'additional_kwargs': {'tool_calls': [{'id': 'call_VI9siBdO7kripnYMKcxoFEsH', 'function': {'arguments': '{}', 'name': 'transfer_to_slide_create_agent'}, 'type': 'function'}], 'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 14, 'prompt_tokens': 382, 'total_tokens': 396, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-C3Yw8grUqZabPnrdRgRUMBEX9Jct2', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, 'type': 'ai', 'name': 'supervisor', 'id': 'run--4c771fc9-28a4-4038-ad61-e3e684d8f812-0', 'example': False, 'tool_calls': [{'name': 'transfer_to_slide_create_agent', 'args': {}, 'id': 'call_VI9siBdO7kr

In [45]:
for chunk in list_chunk:
    if filter_silde_data(chunk):
        print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
        print(chunk.data)
        break

##########  슬라이드 데이터  ##########
{'executor': {'slides': [{'topic': 'jeju_trip', 'idx': 2, 'name': '제주도 인기 관광지', 'description': '제주도에서 가장 인기 있는 관광지를 소개합니다. 자연 경관, 역사적 명소, 현대적인 즐길 거리를 포함합니다.', 'research': True, 'content': '## 제주도 인기 관광지\n\n제주도는 한국에서 가장 인기가 많은 관광지 중 하나로, 그 아름다운 자연 경관과 풍부한 문화적 유산, 현대적인 즐길 거리로 인해 많은 사람들의 사랑을 받고 있습니다. 제주도에서 꼭 방문해야 할 대표적인 자연 경관으로는 여러 곳이 존재합니다.\n\n먼저, 한라산 국립공원은 제주도의 중심부에 자리 잡고 있으며, 한라산은 대한민국 내 가장 높은 산으로 서서히 상승하는 화산 지형이 특징입니다. 이곳은 풍부한 식물과 아름다운 산 전망을 자랑하며, 등산을 즐기는 사람들에게 인기 있는 명소입니다[1][2]. 또 다른 자연 경관으로는 ‘일출봉’으로도 불리는 성산일출봉이 있습니다. 이곳은 약 5,000년 전해저 화산 폭발로 형성된 곳으로, 제주의 자연유산에 차례된 유네스코 세계자연유산장의 일부로 선정되었습니다[3]. 더욱이, 화산 터널인 만장굴 용암 동굴은 용암의 흐름으로 형성된 독특한 자연 경관 중 하나로 잘 알려져 있습니다[7][1]. 이 외에도, 제주도의 아름다움은 천제연 폭포, 중문의 오셀록 박물관, 그리고 비자림 숲 등 자연과 현대적 장소들이 융합되어 있습니다[1][7][2].', 'img_url': 'https://ak-d.tripcdn.com/images/100o1a000001910qs767E_Z_640_10000_R5.jpg_.webp?proc=autoorient&proc=source%2Ftrip', 'design': '⭐ 시작할 슬라이드를 작성하겠습니다.\n- 이 슬라이드는 제주도의 인기 관광지를 강조하며 청중에게 제주도의 자연미와 역사적 

In [49]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(chunk.data['agent']['messages'][-1]['content'])

##########  AI 메시지  ##########
안녕하세요! 슬라이드 생성에 대해 도와드릴 수 있는 내용이 있을까요? 발표 주제나 필요한 내용에 대해서 말씀해 주세요.
##########  AI 메시지  ##########
좋은 선택입니다! 제주도 여행에 관한 슬라이드를 만드는 데 있어 몇 가지 질문을 드릴게요:

1. **슬라이드의 목적**: 이 슬라이드는 개인적인 여행 계획을 위한 것인가요, 아니면 다른 사람에게 제주도를 소개하는 발표용인가요?

2. **포함하고 싶은 내용**: 특정 장소, 음식, 액티비티 등 꼭 다루고 싶은 주제가 있나요?

3. **슬라이드 수**: 몇 장의 슬라이드를 원하시나요?

4. **스타일이나 디자인**: 특별히 원하는 디자인이나 스타일이 있나요? 예를 들면, 밝고 색감 있는 스타일, 심플한 흑백 스타일 등 말이죠.

5. **타겟 audience**: 이 슬라이드를 보는 사람들에 대해 특별히 고려해야 할 사항이 있을까요?

이 정보를 토대로 슬라이드를 제작하겠습니다.
##########  AI 메시지  ##########

##########  AI 메시지  ##########

##########  AI 메시지  ##########
Based on the search results, here is the proposed structure for a report on 제주도 여행 (Jeju Island Travel):

```json
{
  "sections": [
    {
      "topic": "jeju_trip",
      "idx": 2,
      "name": "제주도 인기 관광지",
      "description": "제주도에서 가장 인기 있는 관광지를 소개합니다. 자연 경관, 역사적 명소, 현대적인 즐길 거리를 포함합니다.",
      "research": false,
      "content": "",
      "img": "",
      "html": ""
    },
    {
 

In [47]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("AI :", msg["content"])
        print(msg['name'])

AI : 안녕하세요! 슬라이드 생성에 대해 도와드릴 수 있는 내용이 있을까요? 발표 주제나 필요한 내용에 대해서 말씀해 주세요.
supervisor
AI : 좋은 선택입니다! 제주도 여행에 관한 슬라이드를 만드는 데 있어 몇 가지 질문을 드릴게요:

1. **슬라이드의 목적**: 이 슬라이드는 개인적인 여행 계획을 위한 것인가요, 아니면 다른 사람에게 제주도를 소개하는 발표용인가요?

2. **포함하고 싶은 내용**: 특정 장소, 음식, 액티비티 등 꼭 다루고 싶은 주제가 있나요?

3. **슬라이드 수**: 몇 장의 슬라이드를 원하시나요?

4. **스타일이나 디자인**: 특별히 원하는 디자인이나 스타일이 있나요? 예를 들면, 밝고 색감 있는 스타일, 심플한 흑백 스타일 등 말이죠.

5. **타겟 audience**: 이 슬라이드를 보는 사람들에 대해 특별히 고려해야 할 사항이 있을까요?

이 정보를 토대로 슬라이드를 제작하겠습니다.
supervisor
AI : 
supervisor
AI : Successfully transferred to slide_create_agent
transfer_to_slide_create_agent
AI : 검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
init_planning_agent_node
AI : 생성된 목차는 다음과 같습니다. 

2. jeju_trip - 제주도에서 가장 인기 있는 관광지를 소개합니다. 자연 경관, 역사적 명소, 현대적인 즐길 거리를 포함합니다. 
3. jeju_trip - 제주도 여행 중 꼭 맛봐야 하는 지역 특산 음식과 추천 맛집을 소개합니다. 
4. jeju_trip - 제주도에서 숙박할 수 있는 다양한 형태의 숙소, 호텔, 리조트를 추천합니다. 
5. jeju_trip - 제주도에서 즐길 수 있는 다양한 액티비티와 체험 프로그램을 소개합니다. 
6. jeju_trip - 제주도 여행 시, 준비해야 할 필수 준비물과 체크리스트를 제공합니다. 

generate_sections_

In [ ]:
state["values"]["slides"]